<a href="https://colab.research.google.com/github/RohanC07/Programming_for_DataScience/blob/main/Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This dataset represents the **All India Consumer Price Index (CPI)** for both rural and urban areas up to September 2014. The CPI measures changes in the average price level of a basket of consumer goods and services over time, serving as an important indicator of inflation in the country. The dataset covers various regions, helping to analyze price trends and inflationary pressures in rural and urban sectors. The data is crucial for understanding regional price disparities and evaluating the impact of inflation on different population segments.


**(1)** **Importing the CSV file into the Google Colab!**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')